In [ ]:
!pip install torchtext==0.4

     |████████████████████████████████| 61kB 3.5MB/s 
  Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1:
      Successfully uninstalled torchtext-0.3.1


In [ ]:
import torch
import torchtext
from torchtext.datasets import text_classification
NGRAMS = 2
import os
if not os.path.isdir('./.data'):
    os.mkdir('./.data')
train_dataset, test_dataset = text_classification.DATASETS['AG_NEWS'](
    root='./.data', ngrams=NGRAMS, vocab=None)
BATCH_SIZE = 16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ag_news_csv.tar.gz: 11.8MB [00:00, 47.5MB/s]
120000lines [00:08, 13613.49lines/s]
120000lines [00:19, 6211.28lines/s]
7600lines [00:01, 6402.02lines/s]


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
class TextSentiment(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [ ]:
len(train_dataset.get_vocab())

1308844

In [ ]:
VOCAB_SIZE = len(train_dataset.get_vocab())
EMBED_DIM = 32
NUN_CLASS = len(train_dataset.get_labels())
model = TextSentiment(VOCAB_SIZE, EMBED_DIM, NUN_CLASS).to(device)

In [ ]:
train_dataset.get_labels()

{0, 1, 2, 3}

In [ ]:
def generate_batch(batch):
    label = torch.tensor([entry[0] for entry in batch])
    text = [entry[1] for entry in batch]
    offsets = [0] + [len(entry) for entry in text]

    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text = torch.cat(text)
    return text, offsets, label

In [ ]:
from torch.utils.data import DataLoader

def train_func(sub_train_):

    train_loss = 0
    train_acc = 0
    data = DataLoader(sub_train_, batch_size=BATCH_SIZE, shuffle=True,
                      collate_fn=generate_batch)
    for i, (text, offsets, cls) in enumerate(data):
        optimizer.zero_grad()
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        output = model(text, offsets)
        loss = criterion(output, cls)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        train_acc += (output.argmax(1) == cls).sum().item()

    scheduler.step()

    return train_loss / len(sub_train_), train_acc / len(sub_train_)

def test(data_):
    loss = 0
    acc = 0
    data = DataLoader(data_, batch_size=BATCH_SIZE, collate_fn=generate_batch)
    for text, offsets, cls in data:
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        with torch.no_grad():
            output = model(text, offsets)
            loss = criterion(output, cls)
            loss += loss.item()
            acc += (output.argmax(1) == cls).sum().item()

    return loss / len(data_), acc / len(data_)

In [ ]:
import time
from torch.utils.data.dataset import random_split
N_EPOCHS = 10
min_valid_loss = float('inf')

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=4.0)
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.85)

train_len = int(len(train_dataset) * 0.95)
sub_train_, sub_valid_ = \
    random_split(train_dataset, [train_len, len(train_dataset) - train_len])

for epoch in range(N_EPOCHS):

    start_time = time.time()
    train_loss, train_acc = train_func(sub_train_)
    valid_loss, valid_acc = test(sub_valid_)

    secs = int(time.time() - start_time)
    mins = secs / 60
    secs = secs % 60

    print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
    print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
    print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc * 100:.1f}%(valid)')

Epoch: 1  | time in 0 minutes, 11 seconds
	Loss: 0.0163(train)	|	Acc: 93.7%(train)
	Loss: 0.0000(valid)	|	Acc: 96.1%(valid)
Epoch: 2  | time in 0 minutes, 11 seconds
	Loss: 0.0034(train)	|	Acc: 98.1%(train)
	Loss: 0.0000(valid)	|	Acc: 96.4%(valid)
Epoch: 3  | time in 0 minutes, 11 seconds
	Loss: 0.0021(train)	|	Acc: 99.0%(train)
	Loss: 0.0000(valid)	|	Acc: 96.3%(valid)
Epoch: 4  | time in 0 minutes, 11 seconds
	Loss: 0.0015(train)	|	Acc: 99.3%(train)
	Loss: 0.0000(valid)	|	Acc: 96.1%(valid)
Epoch: 5  | time in 0 minutes, 11 seconds
	Loss: 0.0012(train)	|	Acc: 99.6%(train)
	Loss: 0.0000(valid)	|	Acc: 96.2%(valid)
Epoch: 6  | time in 0 minutes, 12 seconds
	Loss: 0.0010(train)	|	Acc: 99.5%(train)
	Loss: 0.0001(valid)	|	Acc: 95.9%(valid)
Epoch: 7  | time in 0 minutes, 11 seconds
	Loss: 0.0009(train)	|	Acc: 99.7%(train)
	Loss: 0.0001(valid)	|	Acc: 96.1%(valid)
Epoch: 8  | time in 0 minutes, 11 seconds
	Loss: 0.0008(train)	|	Acc: 99.7%(train)
	Loss: 0.0000(valid)	|	Acc: 96.3%(valid)
Epoch: 9

In [ ]:
test_loss, test_acc = test(test_dataset)
print(f'\tLoss: {test_loss:.4f}(test)\t|\tAcc: {test_acc * 100:.1f}%(test)')

Checking the results of test dataset...
	Loss: 0.0004(test)	|	Acc: 90.2%(test)


In [ ]:
import time
from torch.utils.data.dataset import random_split
N_EPOCHS = 10
min_valid_loss = float('inf')

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=4.15)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.86)

train_len = int(len(train_dataset) * 0.95)
sub_train_, sub_valid_ = \
    random_split(train_dataset, [train_len, len(train_dataset) - train_len])

for epoch in range(N_EPOCHS):

    start_time = time.time()
    train_loss, train_acc = train_func(sub_train_)
    valid_loss, valid_acc = test(sub_valid_)

    secs = int(time.time() - start_time)
    mins = secs / 60
    secs = secs % 60

    print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
    print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
    print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc * 100:.1f}%(valid)')

Epoch: 1  | time in 0 minutes, 11 seconds
	Loss: 0.0004(train)	|	Acc: 99.9%(train)
	Loss: 0.0000(valid)	|	Acc: 99.9%(valid)
Epoch: 2  | time in 0 minutes, 11 seconds
	Loss: 0.0005(train)	|	Acc: 99.9%(train)
	Loss: 0.0000(valid)	|	Acc: 99.9%(valid)
Epoch: 3  | time in 0 minutes, 11 seconds
	Loss: 0.0004(train)	|	Acc: 99.9%(train)
	Loss: 0.0000(valid)	|	Acc: 99.9%(valid)
Epoch: 4  | time in 0 minutes, 11 seconds
	Loss: 0.0003(train)	|	Acc: 99.9%(train)
	Loss: 0.0000(valid)	|	Acc: 99.9%(valid)
Epoch: 5  | time in 0 minutes, 11 seconds
	Loss: 0.0002(train)	|	Acc: 99.9%(train)
	Loss: 0.0000(valid)	|	Acc: 99.9%(valid)
Epoch: 6  | time in 0 minutes, 11 seconds
	Loss: 0.0002(train)	|	Acc: 100.0%(train)
	Loss: 0.0000(valid)	|	Acc: 99.9%(valid)
Epoch: 7  | time in 0 minutes, 11 seconds
	Loss: 0.0002(train)	|	Acc: 100.0%(train)
	Loss: 0.0000(valid)	|	Acc: 99.9%(valid)
Epoch: 8  | time in 0 minutes, 11 seconds
	Loss: 0.0001(train)	|	Acc: 100.0%(train)
	Loss: 0.0000(valid)	|	Acc: 99.9%(valid)
Epoch

In [ ]:
print('Checking the results of test dataset...')
test_loss, test_acc = test(test_dataset)
print(f'\tLoss: {test_loss:.4f}(test)\t|\tAcc: {test_acc * 100:.1f}%(test)')

Checking the results of test dataset...
	Loss: 0.0003(test)	|	Acc: 89.5%(test)


In [ ]:
import re
from torchtext.data.utils import ngrams_iterator
from torchtext.data.utils import get_tokenizer

ag_news_label = {0 : "World",
                 1 : "Sports",
                 2 : "Business",
                 3 : "Sci/Tec"}

def predict(text, model, vocab, ngrams):
    tokenizer = get_tokenizer("basic_english")
    with torch.no_grad():
        text = torch.tensor([vocab[token]
                            for token in ngrams_iterator(tokenizer(text), ngrams)])
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item()

ex_text_str = "last match sachin scored 100 runs"

vocab = train_dataset.get_vocab()
model = model.to("cpu")

print("This is a %s news" %ag_news_label[predict(ex_text_str, model, vocab, 2)])

This is a Sports news
